
### **Reddit Segment** 
---



In [1]:
import numpy as np
import pandas as pd
import requests
import json
import csv
import time
import datetime
import os

In [2]:
# def get_pushshift_data(query, after, before, sub):
def get_pushshift_data(query, after, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title=' + str(query) + '&size=1000&after=' + str(
        after) + '&subreddit=' + str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']


In [3]:
def collect_sub_data(subm):
    sub_data = list()  # list to store data points
    title = subm['title']
    url = subm['url']
    try:
        # if flair is available then get it, else set 'NaN'
        flair = subm['link_flair_text']
    except KeyError:
        flair = 'NaN'
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    try:
        # if selftext is available then get it, else set it empty
        selftext = subm['selftext']
        list_of_empty_markers = ['[removed]', '[deleted]']
        # many times selftext would be removed or deleted, if thats the case then set it empty
        if selftext in list_of_empty_markers:
            selftext = ''
    except:
        selftext = ''
    created = datetime.datetime.fromtimestamp(subm['created_utc'])  # 1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    sub_data.append((sub_id, title, selftext, url, author, score, created, numComms, permalink, flair))
    sub_stats[sub_id] = sub_data

In [4]:
def write_subs_to_file():
    
    upload_count = 0
    
    columns = ['post_id', 'title', 'selftext', 'url', 'author', 'score', 'publish_date', 'num_of_comments','permalink', 'flair']
    # print("Ankur")
    df_reddit1 = pd.DataFrame(columns = columns)
    

    for sub in sub_stats:
      li =list(sub_stats[sub][0])
      # print("Shape",type(li),li[0])
      # df_reddit2 = pd.DataFrame(li,columns=['post_id', 'title', 'selftext', 'url', 'author', 'score', 'publish_date', 'num_of_comments','permalink', 'flair'])
      # print(df_reddit2)
      # df_reddit_1.append(df_reddit2,ignore_index = True)
      
      # df_reddit1.iloc[upload_count] = li
      
      # print(type(li),len(li),li)
      # print(df_reddit1)

      di1 = dict.fromkeys(columns,'')
      di1['post_id'],di1['title'],di1['selftext'],di1['url'],di1['author'],di1['score'],di1['publish_date'],di1['num_of_comments'],di1['permalink'],di1['flair'] = li[0],li[1],li[2],li[3],li[4],li[5],li[6],li[7],li[8],li[9]
      # print(di1)
      df_reddit1 = df_reddit1.append(di1, ignore_index = True)
      upload_count +=1
      # print(df_reddit1.shape,upload_count)
      
    
    print(df_reddit1.columns)
   
    return df_reddit1

### **Reddit Sentiment**

In [5]:
!pip install flair
import flair

     |████████████████████████████████| 319 kB 5.2 MB/s 
     |████████████████████████████████| 981 kB 28.5 MB/s 
     |████████████████████████████████| 1.2 MB 37.3 MB/s 
     |████████████████████████████████| 48 kB 5.6 MB/s 
     |████████████████████████████████| 2.9 MB 28.9 MB/s 
     |████████████████████████████████| 788 kB 17.4 MB/s 
     |████████████████████████████████| 19.7 MB 43 kB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 62 kB 746 kB/s 
     |████████████████████████████████| 3.3 MB 42.3 MB/s 
     |████████████████████████████████| 895 kB 36.6 MB/s 
     |████████████████████████████████| 636 kB 36.2 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=e97aa5daa3e537dbadd088a630d1203fea45d700572ddc5

In [6]:
# now sentiment analysis and bucketizing
import nltk
nltk.download('vader_lexicon')
import pandas as pd

from textblob import TextBlob
import os
import datetime
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [7]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
fmt = '%Y-%m-%d %H:00:00'
sid = SentimentIntensityAnalyzer()

2021-10-10 04:24:38,645 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpj1triei9


100%|██████████| 265512723/265512723 [00:12<00:00, 21415751.99B/s]

2021-10-10 04:24:51,425 copying /tmp/tmpj1triei9 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-10-10 04:24:52,634 removing temp file /tmp/tmpj1triei9
2021-10-10 04:24:54,266 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
def get_sentiment_val_for_flair(sentiments):
    """
    parse input of the format [NEGATIVE (0.9284018874168396)] and return +ve or -ve float value
    :param sentiments:
    :return:
    """
    total_sentiment = str(sentiments)
    neg = 'NEGATIVE' in total_sentiment
    if neg:
        total_sentiment = total_sentiment.replace('NEGATIVE', '')
    else:
        total_sentiment = total_sentiment.replace('POSITIVE', '')

    total_sentiment = total_sentiment.replace('(', '').replace('[', '').replace(')', '').replace(']', '')

    val = float(total_sentiment)
    if neg:
        return -val
    return val

In [9]:
def get_sentiment_report_reddit(df):
    # df = pd.read_csv('/content/'+input_filename)
    df = df[['title', 'selftext', 'publish_date']]
    df = df.fillna('')

    df['text'] = df['title'] + ' ' + df['selftext']
    df.set_index('publish_date', inplace=True)
    df.drop(['title', 'selftext'], axis=1, inplace=True)

    columns = ['timestamp','reddit_flair','reddit_tb_polarity', 'reddit_tb_subjectivity', 'reddit_sid_pos','reddit_sid_neg', 'reddit_sid_neu', 'reddit_sid_com']
    final_senti_df = pd.DataFrame(columns = columns)
    final_senti_df.set_index('timestamp',inplace = True)


    for row_i, row in df.iterrows():
        tb_sentiment_polarity_dict = dict()
        tb_sentiment_subjectivity_dict = dict()
        flair_sentiment_dict = dict()

        sid_pos_dict = dict()
        sid_neg_dict = dict()
        sid_neu_dict = dict()
        sid_com_dict = dict()

        data = row['text']
        print(row_i)
        print(data[0:15])
        flair_s = flair.data.Sentence(data)
        flair_sentiment.predict(flair_s)
        flair_total_sentiment = flair_s.labels
        flair_val = get_sentiment_val_for_flair(flair_total_sentiment)

        flair_sentiment_dict[str(row_i)] = flair_val
        tb_sentiment_polarity_dict[str(row_i)] = TextBlob(data).sentiment[0]
        tb_sentiment_subjectivity_dict[str(row_i)] = TextBlob(data).sentiment[1]

        ss = sid.polarity_scores(data)
        sid_pos_dict[str(row_i)] = ss['pos']
        sid_neg_dict[str(row_i)] = ss['neg']
        sid_neu_dict[str(row_i)] = ss['neu']
        sid_com_dict[str(row_i)] = ss['compound']

        flair_df = pd.DataFrame.from_dict(flair_sentiment_dict, orient='index', columns=['reddit_flair'])
        flair_df.index.name = 'timestamp'

        tb_polarity_df = pd.DataFrame.from_dict(tb_sentiment_polarity_dict, orient='index',
                                                columns=['reddit_tb_polarity'])
        tb_polarity_df.index.name = 'timestamp'

        tb_subjectivity_df = pd.DataFrame.from_dict(tb_sentiment_subjectivity_dict, orient='index',
                                                    columns=['reddit_tb_subjectivity'])
        tb_subjectivity_df.index.name = 'timestamp'

        sid_pos_df = pd.DataFrame.from_dict(sid_pos_dict, orient='index',
                                            columns=['reddit_sid_pos'])
        sid_pos_df.index.name = 'timestamp'

        sid_neg_df = pd.DataFrame.from_dict(sid_neg_dict, orient='index',
                                            columns=['reddit_sid_neg'])
        sid_neg_df.index.name = 'timestamp'

        sid_neu_df = pd.DataFrame.from_dict(sid_neu_dict, orient='index',
                                            columns=['reddit_sid_neu'])
        sid_neu_df.index.name = 'timestamp'

        sid_com_df = pd.DataFrame.from_dict(sid_com_dict, orient='index',
                                            columns=['reddit_sid_com'])
        sid_com_df.index.name = 'timestamp'

        # di2 = dict.fromkeys(columns,'')
        

        dummy_df = pd.concat([flair_df, tb_polarity_df, tb_subjectivity_df, sid_pos_df, sid_neg_df, sid_neu_df,
        							sid_com_df], axis=1)
        
        frames=[final_senti_df,dummy_df]

        final_senti_df = pd.concat(frames)
        # display(dummy_df)
        # final_senti_df.append(dummy_df)

        # if os.path.exists(output_filename):
        #     keep_header = False
        # else:
        #     keep_header = True

        # final_senti_df.to_csv(output_filename, mode='a', header=keep_header)

    return final_senti_df



In [10]:
def bucketize_sentiment_report(in_df,time_begin,time_end):
    start_date_time_obj = time_begin
    # start_date_time_obj = time_begin - datetime.timedelta(hours=1)
    
    # end_date_time_obj = datetime.datetime(2019, 11, 22, 00)
    
    end_date_time_obj = time_end
    hr1 = datetime.timedelta(hours=1)
    curr_date_time_obj = start_date_time_obj
    # in_df = pd.read_csv("/content/"+input_filename)


    out_dict = dict()

    while curr_date_time_obj <= end_date_time_obj:
        curr_timestamp = curr_date_time_obj.strftime(format=fmt)
        # print(curr_timestamp)
        # create data dict with all possible timestamps and dummy value of reddit_flair
        # reddit_flair is chosen just randomly as a placeholder
        out_dict[curr_timestamp] = 0
        curr_date_time_obj += hr1

    # print(out_dict)
    out_df = pd.DataFrame.from_dict(out_dict, orient='index',
                                    columns=['reddit_flair'])

    # print(out_dict)
    out_df.index.name = 'timestamp'
    # populate more colums
    out_df['reddit_flair_count'] = 0
    out_df['reddit_tb_polarity'] = 0
    out_df['reddit_tb_polarity_count'] = 0
    out_df['reddit_tb_subjectivity'] = 0
    out_df['reddit_tb_subjectivity_count'] = 0
    out_df['reddit_sid_pos'] = 0
    out_df['reddit_sid_neg'] = 0
    out_df['reddit_sid_neu'] = 0
    out_df['reddit_sid_com'] = 0
    out_df['reddit_sid_count'] = 0

    # print(out_df)
    
    in_df.reset_index(inplace = True)
    # print(in_df)
    for i in range(len(in_df)):
        
        timestamp = in_df.loc[i, 'timestamp']
        out_key = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
        # print('Ankur: start')
        # print(out_key)
        # timestamp is current plus few minutes or seconds, so collect all these data in the bucket of next hour
        out_key += hr1
        out_key = out_key.strftime(format='%Y-%m-%d %H:00:00')
        
        # print(out_df)
        
        # add up all values and count how many values we have added. In next pass we would normalize the values
        try:
            
            out_df.loc[out_key, 'reddit_flair'] += in_df.loc[i, 'reddit_flair']
            
            out_df.loc[out_key, 'reddit_flair_count'] += 1
            out_df.loc[out_key, 'reddit_tb_polarity'] += in_df.loc[i, 'reddit_tb_polarity']
            out_df.loc[out_key, 'reddit_tb_polarity_count'] += 1
            out_df.loc[out_key, 'reddit_tb_subjectivity'] += in_df.loc[i, 'reddit_tb_subjectivity']
            
            # print(out_df.loc[out_key, 'reddit_tb_subjectivity_count'])
            
            out_df.loc[out_key, 'reddit_tb_subjectivity_count'] += 1
            
            # print(out_df.loc[out_key, 'reddit_sid_pos'])
            # print(in_df.loc[i, 'reddit_sid_pos'])
            
            out_df.loc[out_key, 'reddit_sid_pos'] += in_df.loc[i, 'reddit_sid_pos']
            out_df.loc[out_key, 'reddit_sid_neg'] += in_df.loc[i, 'reddit_sid_neg']
            out_df.loc[out_key, 'reddit_sid_neu'] += in_df.loc[i, 'reddit_sid_neu']
            out_df.loc[out_key, 'reddit_sid_com'] += in_df.loc[i, 'reddit_sid_com']
            out_df.loc[out_key, 'reddit_sid_count'] += 1
            
            # print("end")
        except Exception as e:
            # print(e)
            pass
    print('Ankur: end')
    # make timestamp as a column and reindex the dataframe to make loc method happy
    out_df['timestamp'] = out_df.index
    out_df.index = range(len(out_df))

    for i in range(len(out_df)):
        #print(out_df.loc[i, 'timestamp'])
        # normalize the values
        if out_df.loc[i, 'reddit_flair_count'] == 0:
            out_df.loc[i, 'reddit_flair'] = 0
        else:
            out_df.loc[i, 'reddit_flair'] /= out_df.loc[i, 'reddit_flair_count']

        if out_df.loc[i, 'reddit_tb_polarity_count'] == 0:
            out_df.loc[i, 'reddit_tb_polarity'] = 0
        else:
            out_df.loc[i, 'reddit_tb_polarity'] /= out_df.loc[i, 'reddit_tb_polarity_count']

        if out_df.loc[i, 'reddit_tb_subjectivity_count'] == 0:
            out_df.loc[i, 'reddit_tb_subjectivity'] = 0
        else:
            out_df.loc[i, 'reddit_tb_subjectivity'] /= out_df.loc[i, 'reddit_tb_subjectivity_count']

        if out_df.loc[i, 'reddit_sid_count'] == 0:
            out_df.loc[i, 'reddit_sid_pos'] = 0
            out_df.loc[i, 'reddit_sid_neg'] = 0
            out_df.loc[i, 'reddit_sid_neu'] = 0
            out_df.loc[i, 'reddit_sid_com'] = 0
        else:
            out_df.loc[i, 'reddit_sid_pos'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_neg'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_neu'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_com'] /= out_df.loc[i, 'reddit_sid_count']

        # if os.path.exists(output_filename):
        #     keep_header = False
        # else:
        #     keep_header = True

    out_df.drop(['reddit_flair_count', 'reddit_tb_polarity_count', 'reddit_tb_subjectivity_count','reddit_sid_count'], axis=1,
                inplace=True)
    # change back index to timestamp to save the data in csv
    out_df.set_index('timestamp', inplace=True)
    return out_df
    # out_df.to_csv(output_filename)

### **Web Scrapping Segment**

In [11]:
!pip install newspaper3k

     |████████████████████████████████| 211 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 6.4 MB/s 
     |████████████████████████████████| 87 kB 7.1 MB/s 
     |████████████████████████████████| 7.4 MB 29.7 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=aa2f77098be2b21986f670e426d942749db4f4bb941752176cbbd56f11c772d0
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3356 sha256=b3089f83713e60df1b9febc9ba7fc2f12e0818678eae9906ec1159a9738b94d1
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398405 sha256=3a888e008c61100092804145630eb1542b04b65c4457ba089b54c5b6964f1b8a
  Stored in directory: /root/.cache/pip/wheels/4c/91/46/3c208287b726df325a597957432

In [12]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import datetime
import pandas as pd
import time
import os
import numpy as np
import datetime
import pytz

In [13]:
URL = "https://www.google.com/search?q=cryptocurrency+bitcoin+after%3A{after_date}+before%3A{before_date}&hl=en&biw=1536&bih=731&tbs=sbd%3A1&tbm=nws&sxsrf=AOaemvJvsALAxFs9-Zs_bHUW8HthpSttsw%3A1631421050994&ei=eoI9YZicPOCUr7wP9MGg0A8&oq=cryptocurrency+after%3A{after_date}+before%3A{before_date}&gs_l=psy-ab.3...5565.5565.0.6063.1.1.0.0.0.0.156.156.0j1.1.0....0...1c.1.64.psy-ab..0.0.0....0.e0I-wakyIOk"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
    'Content-Type': 'text/html',
}

max_count = 10  # max 10 news articles per day
fmt = '%m/%d/%Y'


In [14]:
def run_google_news_scrapper(**params):

    news_cols = ['date','news_1_text',
             'news_2_text',  'news_3_text',
             'news_4_text', 'news_5_text',
            'news_6_text','news_7_text',
             'news_8_text', 'news_9_text']

    news_data_df = pd.DataFrame(index=[0], columns=news_cols)
    
    for key, value in params.items():
        if key == 'min_date':
            min_date = value
        if key == 'output_file':
            output_file_name = value

    after_date = min_date + datetime.timedelta(days = -1)
    before_date = min_date
    x = str(after_date)
    x = x.split(' ')[0]
    y = str(before_date)
    y = y.split(' ')[0]
    url = URL.format(after_date = x, before_date = y)
    print("this is url:",url)
    print(type(min_date))
    news_data_dict = dict()
    print(min_date)
    news_data_dict['date'] = min_date
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #     print(soup)
    #news_data_dict['status_code'] = response.status_code
    #     columns.append('status_code')
    
    if response.status_code != 200:
        print("* fail *** ")
        return
    # print(response.url)
    #news_data_dict['url'] = response.url

    count = 1

    links = []
    for link in soup.find_all('a'):
        # time.sleep(1)
        link_str = str(link.get('href'))

        # print(link_str)
        # print(link_str.find("url?q"),"  ",link_str)
        
        if (link_str.find("google.com") != -1): continue
        if (link_str.find("coindesk.com") != -1): continue
        if (link_str.find("bloomberg.com") != -1): continue
        no = link_str.find("url?q")
        if no != -1:
            ind = link_str.find("&sa=U&ved")
            links.append(link_str[7:ind])

    links = set(links)
    links = list(links)
    for link_str in links:
        # link_str = str(link.get('url'))
        # print(link_str)
        try:
           
                article = Article(link_str)
                article.download()
                
                article.parse()
                #print(link_str)
                #                 print(article.authors)
                #                 print(article.publish_date)
                #                 print(article.text)
                news_count = 'news_' + str(count)

                #news_data_dict[news_count + '_url'] = link_str
                news_data_dict[news_count + '_text'] = article.text
                
                count += 1
                if count >= max_count:
                    break

        except:
            pass
       
    news_data_df = news_data_df.append(news_data_dict, ignore_index=True)
    


    # if os.path.exists(output_file_name):
    #     keep_header = False
    # else:
    #     keep_header = True
    # news_data_df.to_csv(output_file_name, mode='a', header=keep_header)


#     news_data_df.to_csv(output_file_name)



    return news_data_dict




In [15]:
def google_news_scrapper(start_date_inp, end_date_inp):
    step_obj = datetime.timedelta(days=1)
    # start_date_time_obj = datetime.datetime(2019, 11, 20)
    # end_date_time_obj = datetime.datetime(2021, 9, 11)

    start_date_time_obj = start_date_inp
    end_date_time_obj = end_date_inp
    #start_date_time_obj = start_date_time_obj.replace(tzinfos=pytz.UTC)
    #end_date_time_obj = end_date_time_obj.replace(tzinfos=pytz.UTC)


    news_cols = ['date','news_1_text',
             'news_2_text',  'news_3_text',
             'news_4_text', 'news_5_text',
            'news_6_text','news_7_text',
             'news_8_text', 'news_9_text']

    news_data_df = pd.DataFrame(index=[0], columns=news_cols)
    
    while start_date_time_obj <= end_date_time_obj:
        start_date = start_date_time_obj
        print(start_date)
        dummy_df_2 = run_google_news_scrapper(min_date=start_date, max_date=start_date)
        news_data_df = news_data_df.append(dummy_df_2, ignore_index=True)
        time.sleep(np.random.randint(2, 5))
        start_date_time_obj += step_obj
    # news_data_df.to_csv(output_file_name)

    return news_data_df


In [16]:
def sort_news_report(input_file_name, cleaned_output_file_name, save_index=False):
    df = pd.read_csv(input_file_name)
    df = df.set_index('date', drop=True)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index().drop_duplicates(keep='first')
    df.to_csv(cleaned_output_file_name)
    if save_index:
        df_i = pd.DataFrame(df.index)
        df_i.to_csv(cleaned_output_file_name[0:-4] + '_index.csv')


In [17]:
def clean_news_report(input_df,  save_index=False):
    master_df = input_df
    # get only given columns
    master_df = master_df[
        ['date', 'news_1_text', 'news_2_text', 'news_3_text', 'news_4_text', 'news_5_text', 'news_6_text',
         'news_7_text', 'news_8_text', 'news_9_text']]
    master_df = master_df.set_index('date', drop=True)
    #master_df.index = pd.to_datetime(master_df.index, format=fmt)
    # soft and drop duplicates
    master_df = master_df.sort_index().drop_duplicates(keep='first')
    #idx = np.unique(master_df.index, return_index=True)[1]
    #master_df = master_df.iloc[idx]
    #     master_df.to_csv(cleaned_output_file_name)

    # master_df.to_csv(cleaned_output_file_name)
    return master_df
    if save_index:
        df_i = pd.DataFrame(master_df.index)
        df_i.to_csv(cleaned_output_file_name[0:-4] + '_index.csv')


### **Web Scrapping - newz - sentiment**

In [18]:
import pandas as pd
import flair
from textblob import TextBlob
import os
import datetime
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer

fmt = '%Y-%m-%d'

In [19]:
def get_sentiment_val_for_flair(sentiments):
    """
    parse input of the format [NEGATIVE (0.9284018874168396)] and return +ve or -ve float value
    :param sentiments:
    :return:
    """
    total_sentiment = str(sentiments)
    print(total_sentiment)
    neg = 'NEGATIVE' in total_sentiment
    if neg:
        total_sentiment = total_sentiment.replace('NEGATIVE', '')
    else:
        total_sentiment = total_sentiment.replace('POSITIVE', '')

    total_sentiment = total_sentiment.replace('(', '').replace('[', '').replace(')', '').replace(']', '')

    print(total_sentiment)
    if total_sentiment == '':
      val=0
    else:
      val = float(total_sentiment)
    if neg:
        return -val
    return val

In [20]:
def add_to_dict(final_dict, input_dict):
    """
    add matching key values and store final result in final_dict
    :param final_dict:
    :param input_dict:
    :return:
    """
    for item in final_dict:
        input_dict_val = input_dict.get(item, 0)
        final_dict[item] += input_dict_val

In [21]:
def devide_dict_by_scaler(in_dict, val):
    """
    devide each value of dict by scaler
    :param in_dict:
    :param val:
    :return:
    """
    for item in in_dict:
        in_dict[item] /= val


In [22]:
def get_sentiment_report(data_df, start_date=None, simulate=False):
    """

    :param data_df: input data is panda dataframe, with index as date of the format fmt
    :return: another dataframe with same index as input dataframe and new columns as sentiment values
    """
    # data_df = pd.read_csv(input_filename, index_col=0)
    data_df.reset_index(inplace=True)
    # display(data_df)
    data_df = data_df.dropna(subset=['date'])
    data_df.set_index('date',inplace=True)
    # print(data_df.index)

    col = data_df.columns

    # sid = SentimentIntensityAnalyzer()
    if simulate:
        flair_sentiment = None
        sid = None
    else:
        flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
        sid = SentimentIntensityAnalyzer()
    temp_c = 0
    for row_i, row in data_df.iterrows():
        temp_c += 1
        # print(row_i)

        if start_date is not None:
            start_date_time_obj = datetime.datetime.strptime(start_date, fmt)
            current_date_time_obj = datetime.datetime.strptime(str(row_i), fmt)

            if current_date_time_obj < start_date_time_obj:
                print('Skipping record of date ', str(current_date_time_obj), ' But looking for ',
                      str(start_date_time_obj))
                continue

        total_sentiment_data_count = 0
        tb_sentiment_polarity_dict = dict()
        tb_sentiment_subjectivity_dict = dict()
        flair_sentiment_dict = dict()

        sid_pos_dict = dict()
        sid_neg_dict = dict()
        sid_neu_dict = dict()
        sid_com_dict = dict()

        flair_sentiment_total = 0

        tb_polarity_total = 0
        tb_subjectivity_total = 0

        sid_pos_total = 0
        sid_neg_total = 0
        sid_neu_total = 0
        sid_com_total = 0

        # sid_polarity_total = {'neg': 0., 'neu': 0., 'pos': 0., 'compound': 0.}

        for col_i in range(len(col)):
            data = (str(row[col_i]))
            # print('\t', col_i)
            if data == 'NaN':
                continue

            if simulate:
                flair_sentiment_total = 5
                tb_polarity_total = 6
                tb_subjectivity_total = 7
                total_sentiment_data_count = 9
            else:
                tb_polarity_total += TextBlob(data).sentiment[0]
                tb_subjectivity_total += TextBlob(data).sentiment[1]

                flair_s = flair.data.Sentence(data)
                flair_sentiment.predict(flair_s)
                flair_total_sentiment = flair_s.labels
                flair_val = get_sentiment_val_for_flair(flair_total_sentiment)
                flair_sentiment_total += flair_val

                ss = sid.polarity_scores(data)
                sid_pos_total += ss['pos']
                sid_neg_total += ss['neg']
                sid_neu_total += ss['neu']
                sid_com_total += ss['compound']

                total_sentiment_data_count += 1

        print(str(row_i), ' ', temp_c)
        flair_sentiment_dict[str(row_i)] = flair_sentiment_total / total_sentiment_data_count
        tb_sentiment_polarity_dict[str(row_i)] = tb_polarity_total / total_sentiment_data_count
        tb_sentiment_subjectivity_dict[str(row_i)] = tb_subjectivity_total / total_sentiment_data_count
        print(flair_sentiment_dict[str(row_i)], tb_sentiment_polarity_dict[str(row_i)],
              tb_sentiment_subjectivity_dict[str(row_i)])

        flair_df = pd.DataFrame.from_dict(flair_sentiment_dict, orient='index', columns=['gnews_flair'])
        flair_df.index.name = 'date'

        tb_polarity_df = pd.DataFrame.from_dict(tb_sentiment_polarity_dict, orient='index',
                                                columns=['gnews_tb_polarity'])
        tb_polarity_df.index.name = 'date'

        tb_subjectivity_df = pd.DataFrame.from_dict(tb_sentiment_subjectivity_dict, orient='index',
                                                    columns=['gnews_tb_subjectivity'])
        tb_subjectivity_df.index.name = 'date'

        sid_pos_dict[str(row_i)] = sid_pos_total / total_sentiment_data_count
        sid_neg_dict[str(row_i)] = sid_neg_total / total_sentiment_data_count
        sid_neu_dict[str(row_i)] = sid_neu_total / total_sentiment_data_count
        sid_com_dict[str(row_i)] = sid_com_total / total_sentiment_data_count

        sid_pos_df = pd.DataFrame.from_dict(sid_pos_dict, orient='index',
                                            columns=['gnews_sid_pos'])
        sid_pos_df.index.name = 'timestamp'

        sid_neg_df = pd.DataFrame.from_dict(sid_neg_dict, orient='index',
                                            columns=['gnews_sid_neg'])
        sid_neg_df.index.name = 'timestamp'

        sid_neu_df = pd.DataFrame.from_dict(sid_neu_dict, orient='index',
                                            columns=['gnews_sid_neu'])
        sid_neu_df.index.name = 'timestamp'

        sid_com_df = pd.DataFrame.from_dict(sid_com_dict, orient='index',
                                            columns=['gnews_sid_com'])
        sid_com_df.index.name = 'timestamp'

        final_senti_df = pd.concat([flair_df, tb_polarity_df, tb_subjectivity_df, sid_pos_df, sid_neg_df,
                            sid_neu_df, sid_com_df], axis=1)

        # if os.path.exists(output_filename):
        #     keep_header = False
        # else:
        #     keep_header = True

        # final_senti_df.to_csv(output_filename, mode='a', header=keep_header)

    return final_senti_df


In [23]:
def clean_sentiment_report(master_df):
    # drop duplicates and sort
    # master_df = pd.read_csv(input_filename, index_col=0)
    master_df.index = pd.to_datetime(master_df.index)
    idx = np.unique(master_df.index, return_index=True)[1]
    master_df = master_df.iloc[idx]
    return master_df

### **Crypto Data**

In [24]:
!pip install python-binance

     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 288 kB 10.4 MB/s 
     |████████████████████████████████| 1.3 MB 37.0 MB/s 
     |████████████████████████████████| 214 kB 71.2 MB/s 
     |████████████████████████████████| 103 kB 57.6 MB/s 
     |████████████████████████████████| 160 kB 57.3 MB/s 
     |████████████████████████████████| 271 kB 59.3 MB/s 


In [25]:
import pandas as pd
import os
import time
from binance.client import Client

import pytz
import numpy as np
import datetime
from datetime import timedelta

In [26]:
epoch = 0
fmt = "%Y-%m-%d %H:%M:%S"  # e.g. 2019-11-16 23:16:15
org_columns = ['open',
               'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
               'trades', 'tb_base_av', 'tb_quote_av', 'ignore']

columns_of_interest = ['open', 'high', 'low', 'close', 'volume']


In [27]:
def init_mod():
    # refer: https://www.binance.com/en/support/articles/360002502072 for API keys
    binance_api_key = "786lfq19MTn1G7cB4mozqbjuvd3u2Fl9I0J0spsnflf3SlP3yZIwEdMBANdHN8lV"
    binance_api_secret = "uuAmPtxoxOtqE1wi4KCEzEZZHnATo8pwbIsa8LExM1Y7vGBuaC0kapL7ut4YRfwF"
    binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)
    global epoch
    epoch = datetime.datetime.utcfromtimestamp(0)
    return binance_client

In [28]:
def convert_time_to_utc(pst_time):
    utc = pytz.utc
    pst = pytz.timezone('America/Los_Angeles')
    datetime1 = datetime.datetime.strptime(pst_time, fmt)
    pst_time = pst.localize(datetime1)
    return pst_time.astimezone(utc).strftime(fmt)


def convert_time_to_pst(utc_time):
    datetime_obj = datetime.datetime.strptime(utc_time, fmt)
    return datetime_obj.replace(tzinfo=time.timezone('UTC')).strftime(fmt)


def to_unixmillis(from_date):
    from_date_obj = datetime.datetime.strptime(from_date, fmt)
    past = datetime.datetime(1970, 1, 1, tzinfo=from_date_obj.tzinfo)
    return int((from_date_obj - past).total_seconds() * 1000.0)


def to_datetime(ms):
    return datetime.datetime.fromtimestamp(int(float(ms) / 1000.0))


In [29]:
def download_data_from_binance(symbol, from_date, to_date,  step=0, pause=-1, simulate=False):
    """

    :param symbol:
    :param from_date:
    :param to_date:
    :param output_filename:
    :param step: step in number of days. Download data in batches of days given by 'step'
    :param pause: pause seconds before downloading next batch.
        if pause == -1 --> random sleep(2,5)
        if pause == 0 --> no sleep
        if pause == num--> sleep for num of seconds
    :param simulate:
    :return:
    """
    binance_client = init_mod()
    from_date_obj = datetime.datetime.strptime(from_date, fmt)
    # from_date_obj = from_date
    step_date_obj = from_date_obj + timedelta(days=step)
    step_date = step_date_obj.strftime(fmt)

    from_millis = to_unixmillis(from_date)
    to_millis = to_unixmillis(to_date)
    step_millis = to_unixmillis(step_date)

    count = 0
    while True:
        from_millis_str = str(from_millis)
        step_millis_str = str(step_millis)
        print('Step %d:Downloading data from %s to %s' % (count,
                                                          str(to_datetime(from_millis_str)),
                                                          str(to_datetime(step_millis_str))
                                                          ))
        if not simulate:
            # download data

            klines = binance_client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR,
                                                          from_millis_str, end_str=step_millis_str)
            klines_len = len(klines)
            if klines_len == 0:
                print('\t Failed to download from %s to %s. Got %d' % (str(to_datetime(from_millis_str)),
                                                                       str(to_datetime(step_millis_str)), klines_len
                                                                       ))
                time.sleep(5)

            print('\t Downloaded data of len %d from %s to %s' % (klines_len,
                                                                  str(to_datetime(from_millis_str)),
                                                                  str(to_datetime(step_millis_str))
                                                                  ))
            new_columns = [item + '_' + symbol for item in org_columns]
            new_columns.insert(0, 'timestamp')

            data_df = pd.DataFrame(klines,
                                   columns=new_columns)
            data_df['timestamp'] = pd.to_datetime(data_df['timestamp'], unit='ms')
            data_df.set_index('timestamp', inplace=True)
            # data_df.to_csv(output_filename)
            # display(data_df)
            return  data_df 

        # move to next step of batches
        from_millis = step_millis
        step_date_obj = step_date_obj + timedelta(days=step)
        step_date = step_date_obj.strftime(fmt)
        step_millis = to_unixmillis(step_date)
        count = count + 1
        if pause == -1:
            pause = np.random.randint(2, 5)
        time.sleep(pause)
        if step_millis >= to_millis:
            break
           


In [30]:
def concat_binance_data(ltc,eth,btc):
    df_list = []
    # for num, symbol in enumerate(symbol_list):
    #     filename = str('%s-binance-data.csv' % (symbol))
    #     df = pd.read_csv(filename, index_col=0)
    #     df_list.append(df)
    df_list.append(ltc)
    df_list.append(eth)
    df_list.append(btc)

    result = pd.concat(df_list, axis=1, sort=True)
    result.index = pd.to_datetime(df.index)
    result = result.sort_index().drop_duplicates(keep='first')
    idx = np.unique(result.index, return_index=True)[1]
    result = result.iloc[idx]

    new_columns = [item + '_' + 'BTCUSDT' for item in columns_of_interest]
    # new_columns.insert(0, 'timestamp')

    for num, symbol in enumerate(symbol_list):
        if symbol == 'BTCUSDT':
            continue
        new_columns.append('close_' + symbol)
        new_columns.append('volume_' + symbol)

    result = result[new_columns]
    # result.to_csv(output_filename)
    return result

In [31]:
def remove_dup_by_index(output_filename):
    result = pd.read_csv(output_filename, index_col=0)
    result.index = pd.to_datetime(result.index)
    result = result.sort_index()        #.drop_duplicates(keep='first')
    idx = np.unique(result.index, return_index=True)[1]
    result = result.iloc[idx]
    result.to_csv(output_filename)

In [32]:
def append_binance_data(master_output_filename, concat_output_filename):
    master_df = pd.read_csv(master_output_filename)
    new_df = pd.read_csv(concat_output_filename)
    master_df = master_df.append(new_df)
    master_df.set_index('timestamp', inplace=True)
    master_df.index = pd.to_datetime(master_df.index)
    master_df = master_df.sort_index().drop_duplicates(keep='first')
    master_df.to_csv(master_output_filename)

### **Merge dataframes**

In [33]:
import pandas as pd
import datetime

In [34]:
def list_diff(list1, list2):
    """"
    returns list1 - list2 and list2 - list1
    to find difference between 2 lists.
    """
    l1_l2 = []
    l2_l1 = []
    for ele in list1:
        if not ele in list2:
            l1_l2.append(ele)

    for ele in list2:
        if not ele in list1:
            l2_l1.append(ele)

    return l1_l2, l2_l1


In [35]:
def merge_crypto_gnews_sentiment(crypto_data,gnews_data):
    """

    News data is per day, and crypto data is per hour.
    So news data is replicated 24 times for each hour w.r.t. each day so that it can be concatinated with crypto data

    :param crypto_data_filename:
    :param google_news_data_filename:
    :param output_data_filename:
    :return:
    """
    # crypto_df = pd.read_csv(crypto_data_filename, index_col=0)
    crypto_df = crypto_data
    crypto_df.index = pd.to_datetime(crypto_df.index)
    # news_df = pd.read_csv(google_news_data_filename, index_col=0)
    news_df = gnews_data
    # print(news_df.index)
    ilist = [str(d) for d in news_df.index]
    
    # print(type(ilist[0]))
    
    hr1 = timedelta(hours=1)
    news_col = list(news_df.columns)
    news_df_comb = pd.DataFrame(columns=news_col)

    print('enu',enumerate(ilist))
    for ilist_index, ilist_item in enumerate(ilist):
        
        dt = datetime.datetime.strptime(ilist_item, '%Y-%m-%d %H:%M:%S')
        # dt = datetime.datetime.strptime(ilist_item, '%d-%m-%Y')
        for hr_ in range(0, 24):
            row_id_new = dt.strftime('%Y-%m-%d %H:00:00')
            row_id_value = news_df.loc[ilist_item]
            dt += hr1
            news_df_comb.loc[row_id_new] = row_id_value
    news_df_comb.index.name = 'timestamp'
    # news_df_comb.to_csv(google_news_data_filename[0:-4] + '_with_timestamp.csv')
    result = pd.concat([crypto_df, news_df_comb], axis=1)
    # result.to_csv(output_data_filename)

    return news_df_comb,result

In [36]:
def merge_crypto_gnews_reddit_sentiment(crypto_gnews, reddit_data):
    # display(crypto_gnews)
    # display( reddit_data)
    
    # print(crypto_gnews.index)
    # print(reddit_data.index)
    # crypto_gnews_df = pd.read_csv(crypto_gnews_filename, index_col=0)
    crypto_gnews_df = crypto_gnews

    # reddit_df = pd.read_csv(reddit_data_filename, index_col=0)
    reddit_df = reddit_data
    reddit_df.index = pd.to_datetime(reddit_df.index)
    
    # print(crypto_gnews.index)
    # print(reddit_data.index)
    
    result = pd.concat([crypto_gnews_df, reddit_df], axis=1)
   
    # result.to_csv(crypto_gnews_reddit_filename)
    return result

### **run function**

In [57]:
import datetime
import pandas as pd
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import math
import time
import datetime
from dateutil import parser
import matplotlib.dates as mdates
import matplotlib.ticker as plticker

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import keras

from numpy import newaxis

import time

import firebase_admin
from firebase_admin import credentials,db,firestore

from google.colab import drive
drive.mount('drive')



Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [37]:
def create_dataset(dataset, pred_col, look_back=1):
  dataX, dataY = [], []
  
  for i in range(len(dataset)-look_back):
    a = dataset[i:(i+look_back), :]
    dataX.append(a)
    dataY.append(dataset[i + look_back, pred_col])
  
  return np.array(dataX), np.array(dataY)


In [62]:


def run_script():

  date_today = datetime.date.today()
  date_yday = date_today - datetime.timedelta(days = 1)
  time_now = datetime.datetime.now()
  this_hour = time_now.replace(minute=0,second=0,microsecond=0)
  
  last_hour = this_hour - datetime.timedelta(hours =1)
  print('Time Now ',time_now)
  print('Last Hour',last_hour)
  print('date_yday ',date_yday)
  print('this hour ',this_hour)
  
  sub_reddit = 'bitcoin'
  key_word = 'bitcoin'
  after = str(int(last_hour.timestamp()))
  
  global sub_stats
  sub_count = 0
  sub_stats = {}
  
  reddit_data = get_pushshift_data(key_word, after, sub_reddit)
  print('reddit data ',reddit_data)
  max_count = 100
  count = 0
  
  while len(reddit_data) > 0 and count < max_count:
    print('count ', count)
    for submission in reddit_data:
      collect_sub_data(submission)
      sub_count += 1
    
    print(len(reddit_data))
    print(str(datetime.datetime.fromtimestamp(reddit_data[-1]['created_utc'])))
    after = reddit_data[-1]['created_utc']
    reddit_data = get_pushshift_data(key_word, after, sub_reddit)
    count = count + 1
    
    
  df_reddit=write_subs_to_file()
  df_reddit_sentiment = get_sentiment_report_reddit(df_reddit)
  
  df_reddit_sentiment_bucketized = bucketize_sentiment_report(df_reddit_sentiment,last_hour,time_now)
  print(pd.DataFrame())
  
  
  df_news = google_news_scrapper(date_yday, date_today)
  
  df_news_clean = clean_news_report(df_news)
  
  df_news_clean_senti = get_sentiment_report(df_news_clean, simulate=False)
  df_news_clean_senti_final = clean_sentiment_report(df_news_clean_senti)
  
  global symbol_list
  symbol_list = ['LTCUSDT', 'ETHUSDT', 'BTCUSDT']
  
  from_date = last_hour.strftime('%Y-%m-%d %H:00:00')
  
  to_date = time_now.strftime('%Y-%m-%d %H:00:00')
  
  
  for num, symbol in enumerate(symbol_list):
    output_filename = '%s-binance-data.csv' % (symbol)
    print('-' * 60)
    print('Downloading data from %s to %s for %s' % (last_hour, to_date, symbol))
    print('-' * 60)
    
    global df
    df = download_data_from_binance(symbol, from_date, to_date,  step=1, pause=-1, simulate=False)
    if symbol == 'LTCUSDT':
      df_LTC = df
    if symbol == 'ETHUSDT':
      df_ETH = df
    if symbol == 'BTCUSDT':
      df_BTC = df
      
  df_concat_crypto = concat_binance_data(df_LTC,df_ETH,df_BTC)
  
  df_check,df_crypto_gnews = merge_crypto_gnews_sentiment(df_concat_crypto, df_news_clean_senti_final)
  
  final_df = merge_crypto_gnews_reddit_sentiment(df_crypto_gnews, df_reddit_sentiment_bucketized)
  final_df = final_df.dropna()
  
  df_csv = pd.read_csv('/content/drive/MyDrive/bitpredict/final.csv',index_col=0)
  print('df_csv.columns',df_csv.columns,' len(df_csv.columns) ',len(df_csv.columns))
  li_data = list(final_df.loc[this_hour])
  
  for i in range(len(li_data)):
    li_data[i] = round(float(li_data[i]),4)
    
  print(len(li_data),len(df_csv.columns),li_data)
  index = this_hour.strftime('%d-%m-%Y %H:%M')
  print('index',index)
  
  df_csv.loc[index] = li_data

  print('df_csv.columns',df_csv.columns,' len(df_csv.columns) ',len(df_csv.columns))
  if len(df_csv.columns)==23:
    df_csv.to_csv('final.csv')
    !cp final.csv "drive/My Drive/bitpredict"
  
  org_df = df_csv
  print(org_df.shape)
  
  print(df_csv.iloc[-1])
  
  org_df['flair'] = org_df['gnews_flair'] + org_df['reddit_flair']
  org_df['tb_polarity'] = org_df['gnews_tb_polarity'] + org_df['reddit_tb_polarity']
  org_df['tb_subjectivity'] = org_df['gnews_tb_subjectivity'] + org_df['reddit_tb_subjectivity']
  org_df['sid_pos'] = org_df['gnews_sid_pos'] + org_df['reddit_sid_pos']
  org_df['sid_neg'] = org_df['gnews_sid_neg'] + org_df['reddit_sid_neg']
  org_df = org_df[['open_BTCUSDT','high_BTCUSDT','low_BTCUSDT', 'close_BTCUSDT', 'volume_BTCUSDT', 'close_LTCUSD', 'volume_LTCUSD', 'close_ETHUSD', 'volume_ETHUSD', 'flair', 'tb_polarity', 'tb_subjectivity', 'sid_pos', 'sid_neg']]
  
  print(org_df.iloc[-1])
  print(org_df.shape)
  
  dataset = org_df.values
  dataset = dataset.astype('float32')
  scaler = MinMaxScaler(feature_range=(0, 1))
  dataset = scaler.fit_transform(dataset)
  
  num_of_features = len(org_df.columns)
  print('Number of features', num_of_features)
  
  expr_name = 'expr_4'
  look_back = 24*15 
  lstm_layers = 64
  epochs=5
  batch_size=128
  
  train_size_percent = 0.80
  pred_col = org_df.columns.get_loc('close_BTCUSDT')
  
  # def create_dataset(dataset, pred_col, look_back=1):
  #   dataX, dataY = [], []
    
  #   for i in range(len(dataset)-look_back):
  #     a = dataset[i:(i+look_back), :]
  #     dataX.append(a)
  #     dataY.append(dataset[i + look_back, pred_col])
  #   return np.array(dataX), np.array(dataY)
  
  train_size = int(len(dataset) * train_size_percent)
  test_size = len(dataset) - train_size
  train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
  print('train:',train)
  
  trainX, trainY = create_dataset(train, pred_col, look_back=look_back)
  testX, testY = create_dataset(test, pred_col, look_back=look_back)
  
  trainX = np.reshape(trainX, (trainX.shape[0], look_back, num_of_features))
  testX = np.reshape(testX, (testX.shape[0],look_back, num_of_features))
  
  print('Training dataset length ', len(train))
  print('Testing dataset length ', len(test))
  print('look_back ', look_back)
  print('total data',dataset.shape,' ',len(train)+len(test))
  model = keras.models.load_model('/content/drive/MyDrive/bitpredict/model_l15_e8_train_last_80.h5')
  
  inp = testX[-10][newaxis,:,:]
  print(inp.shape)
  
  testPredict = model.predict(inp)
  print(testPredict,testPredict.shape)
  
  testPredict_extended = np.zeros((len(testPredict),num_of_features))
  
  testPredict_extended[:,pred_col] = testPredict[:,0]
  testPredict = scaler.inverse_transform(testPredict_extended)[:,pred_col]
  
  testY_extended = np.zeros((len(testY),num_of_features))
  testY_extended[:,pred_col]=testY
  testY = scaler.inverse_transform(testY_extended)[:,pred_col]
  
  print('prediction:',testPredict)
  print('Actual',testY[-10])
  
    
  print(testPredict)
  print('Actual',testY[-1:])
  
  db_hour = str(int(time.mktime(this_hour.timetuple())))
  pprize = testPredict[0]
  oprize = testY[-1]
  
  
  if not firebase_admin._apps:
    
    cred_obj = credentials.Certificate('/content/drive/MyDrive/bitpredict/bitcoin-price-predictor-firebase-adminsdk-erp3v-7e3aceca37.json')
    
    default_app = firebase_admin.initialize_app(cred_obj, {
        'projectId': 'bitcoin-price-predictor',
        })
    
  pprize = round(pprize,4)
  oprize = round(oprize,4)
  
  realtimedb = {
          'timestamp':db_hour,
          'predicted':oprize,
          'last':pprize
  }
  
  db = firestore.client()
  doc_ref = db.collection(u'bitsmaptrimmed')
  doc_ref.add(realtimedb)
  print(realtimedb)

### **run file**

In [ ]:
# run_script()
# import datetime
# import time
# # filename='/content/drive/MyDrive/bitpredict/pipelining.py'



while(1):

  try:
    d = datetime.datetime.now().minute
    if d==0:
      run_script()
      f = open('/content/drive/MyDrive/bitpredict/log.txt', 'a')
      print(str(datetime.datetime.now()) + " EXECUTED ")
      f.write(str(datetime.datetime.now()) + " EXECUTED" )
      f.close()
    else:
      print(datetime.datetime.now(),' waiting')
      time.sleep(50)

  except Exception as e:
    f = open('/content/drive/MyDrive/bitpredict/log.txt', 'a')
    print(str(datetime.datetime.now()) + " NOT EXECUTED "+ str(e))
    f.write(str(datetime.datetime.now()) + " NOT EXECUTED "+ str(e))
    f.close()




    # try:
    #     with open(filename, "rb") as source_file:
    #         code = compile(source_file.read(), filename, "exec")

    #     d = datetime.datetime.now().minute
       

    #     # print(d)
    #     #print(d,type(d))
    #     if d==8:
    #         exec(code)
    #         f = open('/content/drive/MyDrive/bitpredict/log.txt', 'a')
    #         f.write(str(datetime.datetime.now())+ " EXECUTED\n ")
    #         f.close()
    #     else:
    #         time.sleep(55)

    # except Exception as e:
    #     f = open('/content/drive/MyDrive/bitpredict/log.txt', 'a')
    #     print()
    #     f.write(str(datetime.datetime.now()) + " NOT EXECUTED "+ str(e))
    #     f.close()


# exec(code, globals, locals)
# # exec(compile(open('pipelining.py', "rb").read(), 'pipelining.py', 'exec'))
# print('this\nankur')


2021-10-10 05:57:52.255414  waiting
2021-10-10 05:58:42.298663  waiting
2021-10-10 05:59:32.306052  waiting
Time Now  2021-10-10 06:00:22.312373
Last Hour 2021-10-10 05:00:00
date_yday  2021-10-09
this hour  2021-10-10 06:00:00
https://api.pushshift.io/reddit/search/submission/?title=bitcoin&size=1000&after=1633842000&subreddit=bitcoin
reddit data  [{'all_awardings': [], 'allow_live_comments': False, 'author': 'Teacherinthestreets', 'author_flair_css_class': 'noob', 'author_flair_richtext': [{'e': 'text', 't': 'redditor for 4 weeks'}], 'author_flair_template_id': '2ec8e69e-6c36-11e9-a04b-0afb553d4ea6', 'author_flair_text': 'redditor for 4 weeks', 'author_flair_text_color': 'dark', 'author_flair_type': 'richtext', 'author_fullname': 't2_ef7wmpjh', 'author_is_blocked': False, 'author_patreon_flair': False, 'author_premium': False, 'awarders': [], 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1633843193, 'domain': 'youtube.com', 'full_link': 'https://www.reddit.com/r/Bitcoi